In [ ]:
import nltk
nltk.download('movie_reviews')
nltk.download('stopwords')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#the following two lines is to load data
#it pops up a separate window, Select corpora, then download Movie Reviews
#import nltk
#nltk.download()

print("Naive Bayes Text classification")

print('step 1: ')
from nltk.corpus import movie_reviews as mr
#print(mr.readme())
# divide the movie_reviews into 2 categories: positive and negative
positive = mr.fileids('pos')
negative = mr.fileids('neg')
# form a new movie_reviews as a dictionary by positive and negative
new_mr = dict(pos = positive, neg = negative)
# the length of positive and negative lists are both 1000
posLen = len(new_mr['pos'])
negLen = len(new_mr['neg'])
print('movie reviews are loaded with ')
print('%s positive reviews ' % posLen)
print('%s negative reviews ' % negLen)
print()


print('step 2: ')
from nltk.corpus import stopwords
sw = stopwords.words('english');
# here in the stopwords list 'sw'
# it should remove some words from the list
# such as: 'no','nor','not','too', and
# "don't","aren't", "couldn't", 'didn', "didn't", 
# "doesn't", "hadn't", "hasn't", "haven't", "isn't", "mightn't",
#  "mustn't", "needn't", "shan't", "shouldn't", "wasn't", "weren't",
#  "won't", "wouldn't"
sw.remove('no')
sw.remove('nor')
sw.remove('not')
sw.remove('too')
sw1 = []
for v in sw:
    if not v.endswith("n't"):
        sw1.append(v)
    #else:
        #print(v)
sw1.remove('ain')
sw1.remove('aren')
sw1.remove('couldn')
sw1.remove('didn')
sw1.remove('doesn')
sw1.remove('hadn')
sw1.remove('haven')
sw1.remove('isn')
sw1.remove('mightn')
sw1.remove('hasn')
sw1.remove('mustn')
sw1.remove('needn')
sw1.remove('shan')
sw1.remove('wasn')
sw1.remove('weren')
sw1.remove('wouldn')
sw = sw1
#print(len(sw))
print('there are %s stopwords loaded' % len(sw))

# input a word, 
# determine whether it is a stopword or not
# remove all punctuations from the word
# to lower case
def pre_process(word, sw):
  # remove punctuations except apostrophe
  w = ""
  hasAlphabet = False
  for i in word:
    if i >= 'a' and i <= 'z':
      w = w + i
      hasAlphabet = True
    elif i >= 'A' and i <= 'Z':
      w = w + i
      hasAlphabet = True
    elif i == '\'': # apostrophe
      w = w + i
  #print(w)
  # check whether it is stopword
  if hasAlphabet and w not in sw:
    return w.lower()
  return ''


# selection the 3000 most important words.
# find the 3000 most important words by frequency
# top, is how many words in the list to be returned
# in this program, top is 2048
def importantWords(corpus, file, top, sw):
  iw = []
  words_dic = {}
  pLen = len(file['pos'])
  i = 0
  while i < pLen:
    movie_review = corpus.words(file['pos'][i])
    for w in movie_review:
      w = pre_process(w, sw)
      if w not in words_dic:
        words_dic[w] = 1
      else:
        words_dic[w] = words_dic[w] + 1
    i = i + 1
  nLen = len(file['neg'])
  i = 0
  while i < nLen:
    movie_review = corpus.words(file['neg'][i])
    for w in movie_review:
      w = pre_process(w, sw)
      if len(w) > 0:
        if w not in words_dic:
          words_dic[w] = 1
        else:
          words_dic[w] = words_dic[w] + 1
      else:
        continue
    i = i + 1
  words_dic.pop('') # remove space
  sorted_words_dic = sorted(words_dic.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)
  # add the top words to the list to be returned
  i = 0
  while i < top:
    iw.append(sorted_words_dic[i][0])
    #print(sorted_words_dic[i])
    i = i + 1
  return iw

# this method finally not used
# focus on the adj. words 
# always end with: 
# and negative words such as: no, nor, not, n't
def focusWords(word):
  adj_list = [ 'ant', 'ary', 'ble', 'cal', 'ed', 'ful', 'ic', 'ish', 'ive', 'lar', 'less', 'ous', 'y']
  for w in adj_list:
      if word.endswith(w):
          return True
  neg_list = ['no', 'nor', 'not', 'too', 'never', 'little']
  for w in neg_list:
      if w == word:
          return True
  if word.endswith("n't"):
      return True
  neg_phrase = ['ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'haven', 'isn', 'mightn', 'hasn', 'mustn', 'needn', 'wasn', 'weren', 'wouldn']
  for w in neg_phrase:
      if word.startswith(w):
          return True
  return False


# iw, the given data is the most-important-word list
# tmp_dic, the given data that represents all the words of a movie review
# generate a certain size dict-type data
# sample: [0:True, 2:False, ..., 2047: True]
# 0, .. , 2047, is the index corresponding to the words stored in the most-important-word list
# True, means the index word is also exsiting in tmp_dic, False means not
def generate_feature(iw, tmp_dic):
  arrayLen = len(iw)
  features = {}
  i = 0
  while i < arrayLen:
    w = iw[i]
    if w in tmp_dic:
      features[i] = True
    else:
      features[i] = False
    i = i + 1
  return features

# select almost every word as a feature
# the input parameter is a list consisting of words
# and return a dictionary which includes useful words after pre-processing
def feature_selection(words, sw, iw):
  wd = {}
  for word in words:
    word = pre_process(word, sw)
    if len(word) > 0:
      wd[word] = True
  wd = generate_feature(iw, wd)
  return wd

# collect several/some/many dict-type positive or negative movie reviews into a list
# for training or testing
# the returned list is composed of tuple
# every tuple is the ict-type positive or negative movie reviews + clf
# corpus, should be the 'mr'
# files, should be the 'new_mr'
# clf, is the classification: pos or neg
def collect_features(corpus, files, startID, endID, clf, sw, iw):
  col = []
  i = startID
  while i < endID:
    dic = feature_selection(corpus.words(files[clf][i]), sw, iw)
    tup = (dic, clf)
    col.append(tup)
    i = i + 1
  return col

# generate a most-important-word list
# that store the top 2048 words by frequency
# and store them in a list
top = 128 * 2 * 2 * 2 * 2 * 2
iw = importantWords(mr, new_mr, top, sw )


# divide the whole set into train and test sets
train_start_id = 0
train_end_id = 900
test_start_id = 900
test_end_id = 1000

# collect train and test sets by using feature selection method
pos_train = collect_features(mr, new_mr, train_start_id, train_end_id, 'pos', sw, iw)
pos_test = collect_features(mr, new_mr, test_start_id, test_end_id, 'pos', sw, iw)
neg_train = collect_features(mr, new_mr, train_start_id, train_end_id, 'neg', sw, iw)
neg_test = collect_features(mr, new_mr, test_start_id, test_end_id, 'neg', sw, iw)

# whole sets
train_set = pos_train + neg_train
test_set = pos_test + neg_test

# shuffle the order of the train and test sets
from sklearn.utils import shuffle
train_set = shuffle( train_set )
test_set = shuffle( test_set )

print('pre-processing: remove punctuations except apostrophe and to lower case')
print('feature selection: geenrate the top %s most-important-word list by frequency' %top)
print('use the most-important-word list to extract features of every movie review, which will be a certain size dict-type data')
print('divide the movie reviews into training and testing set after above processing')
print('training set: 90% \ntesting set: 10%')
print()


print('step 3:')
# text classification
# use Naive Bayes Classifier
from nltk.classify import NaiveBayesClassifier
print('text classification: Naive Bayes Text classification')

# Train a classifier on our training data.
classifier = NaiveBayesClassifier.train(train_set)
print('using NB classifier to train the model on 90% of movie reviews')
print()

In [ ]:
print('step 4:')
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print('evaluate trained model on 10% of movie reviews')

# pairs, after using the model to test, it will generate every testing case as a tuple
# such as: ( 'pos', 'pos' ), the first one is predicted one, and the latter one is actual one
# pos_label, 'pos' or 'neg'
# printout, True means to print it out and False means not
def evaluate(pairs, pos_label, printout):
    predicted, actual = zip(*pairs)
    precision = precision_score(actual,predicted,pos_label=pos_label)
    recall = recall_score(actual,predicted,pos_label=pos_label)
    accuracy = accuracy_score(actual,predicted)
    f1 = f1_score(actual,predicted,pos_label=pos_label)
    if printout:
      print_evaluation(precision, recall, accuracy, f1, pos_label)
    return (precision, recall, accuracy, f1)

def print_evaluation(precision, recall, accuracy, f1, pos_label):
    title =  'Evaluation with pos label = %s' % pos_label
    print(title)
    print('=' * len(title))
    print('{0:10s} {1:.2f} %'.format('Precision',precision*100))
    print('{0:10s} {1:.2f} %'.format('Recall',recall*100))
    print('{0:10s} {1:.2f} %'.format('Accuracy',accuracy*100))
    print('{0:10s} {1:.2f} %'.format('F1 score',f1*100))

pairs = [(classifier.classify(predicted), actual) for (predicted, actual) in test_set]
printout = True
precision, recall, accuracy, f1 = evaluate(pairs, 'pos', printout)
print()
precision, recall, accuracy, f1 = evaluate(pairs, 'neg', printout)
print()


print('step 5:')
print('testing the exclusive movie reviews with this model')
# new testing examples excluded from the move_reviews set
movie_review = "Joining the Criterion Collection as #1060, Amores perros is given the full-treatment for its upcoming release: a 4K restoration, new audio tracks, newly recorded bonus materials, essays, and more."
prediction = classifier.classify(feature_selection(movie_review.split(), sw, iw))
print('moview review is: \t' + movie_review)
print('prediction: \t' + prediction)
movie_review = "I find it hard to believe that anyone managed to get any kind of entertainment from the film itself, lurid or otherwise."
prediction = classifier.classify(feature_selection(movie_review.split(), sw, iw))
print('moview review is: \t' + movie_review)
print('prediction: \t' + prediction)
classifier.show_most_informative_features()

step 4:
evaluate trained model on 10% of movie reviews
Evaluation with pos label = pos
Precision  87.36 %
Recall     76.00 %
Accuracy   82.50 %
F1 score   81.28 %

Evaluation with pos label = neg
Precision  78.76 %
Recall     89.00 %
Accuracy   82.50 %
F1 score   83.57 %

step 5:
testing the exclusive movie reviews with this model
moview review is: 	Joining the Criterion Collection as #1060, Amores perros is given the full-treatment for its upcoming release: a 4K restoration, new audio tracks, newly recorded bonus materials, essays, and more.
prediction: 	neg
moview review is: 	I find it hard to believe that anyone managed to get any kind of entertainment from the film itself, lurid or otherwise.
prediction: 	neg
Most Informative Features
                    1730 = True              pos : neg    =     15.6 : 1.0
                    2976 = True              neg : pos    =     14.2 : 1.0
                    2816 = True              neg : pos    =     11.8 : 1.0
                    2470 =

In [ ]:
print(iw[1730])
print(iw[2976])
print(iw[2816])
print(iw[2470])
print(iw[2700])
print(iw[4004])
print(iw[3894])
print(iw[3950])
print(iw[1306])
print(iw[3169])


outstanding
ludicrous
idiotic
religion
jolie
hudson
insulting
sucks
mulan
stupidity
